In [ ]:
%matplotlib inline
from datetime import date, datetime, timedelta, time
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy

In [ ]:
from simlightcurve.curves import GaussExp as Gred
from simlightcurve.solvers import find_rise_t, find_peak
from simtransient.utils import mahalanobis_sq
from scipy.stats import norm, multivariate_normal
import scipy.optimize as op
import emcee

In [ ]:
seaborn.set_context('poster')
current_palette = seaborn.color_palette()

In [ ]:
## Generic multivariate gaussian
ndim=2
e1=1
e2=1
xcov=0.8
cov = np.asarray([[e1,xcov],
                  [xcov,e2]])
print "Det:", e1*e2-xcov*xcov
# true_mean = np.random.random(size=2)
true_mean=np.array([0,
                    0])
print "True Mean: ", true_mean
rv = multivariate_normal(mean=true_mean, cov=cov)

n_grid=200
half_grid_size = 3*max(e1,e2)
x = np.linspace(true_mean[0]-half_grid_size,true_mean[0]+half_grid_size,n_grid)
y = np.linspace(true_mean[1]-half_grid_size,true_mean[1]+half_grid_size,n_grid)
grid = np.dstack(np.meshgrid(x,y))
# print x[0],x[1]
# y = np.sqrt(x[0]**2+x[1]**2)

seaborn.set_palette('husl')
cmap = plt.cm.get_cmap('afmhot')
plt.contourf(x,y,rv.pdf(grid),25,cmap=cmap)
plt.ylabel('y')
plt.xlabel('x')
plt.axvline(true_mean[0], ls=':')
plt.axhline(true_mean[1], ls=':')
plt.colorbar()

n_samples = 250
data = np.atleast_2d(rv.rvs(n_samples))
# print "data:", data
print "datashape", data.shape
plt.scatter(data[:,0],data[:,1])
# plt.scatter(data[0,:],data[1,:])

pltlims = plt.xlim(), plt.ylim()

OK, now let's check we get something similar when we're transforming from a unit hypercube (or unit-square, in this case):

In [ ]:
ndim=2
nsamples=1000
hypercube_samples = scipy.stats.uniform.rvs(size=ndim*nsamples).reshape(ndim,-1)
hypercube_samples.shape

In [ ]:
n_cube=1000+1
cube_x = np.linspace(0,1,n_cube)
cube_y = np.linspace(0,1,n_cube)
cube = np.dstack(np.meshgrid(cube_x,cube_y))
hypercube_samples = np.rollaxis(cube,2)
hypercube_samples=hypercube_samples.reshape(2,-1)

In [ ]:
cov

In [ ]:
cholesky = scipy.linalg.cholesky(cov,lower=True)
assert (np.dot(cholesky,cholesky.T)==cov).all()


In [ ]:
normal_samples = norm.ppf(hypercube_samples.ravel()).reshape(hypercube_samples.shape)
# plt.hist(normal_samples.ravel())
normal_samples[:,:3]

In [ ]:
# normal_samples=np.array([np.arange(-5,5),
# #                          np.arange(-5,5)
# #                          np.ones(10),
#                          np.zeros(10),
#                         ])
# # normal_samples=np.array([[1],[1]])
# normal_samples

In [ ]:
true_mean = true_mean.reshape(ndim,-1)
true_mean

In [ ]:
hc_cho_data = cholesky.dot(normal_samples) + true_mean
# hypercube_data

In [ ]:
cov

In [ ]:
eigvec, eigval = scipy.linalg.eig(cov)
# a = rot.dot(np.diag(np.sqrt(eig)))
# np.isclose(a.dot(a.T), cov).all()
# a = eigvec.dot(np.diag(np.sqrt(eigval)))
# eigval

In [ ]:
# hc_svd_data=a.dot(normal_samples) + true_mean

In [ ]:

seaborn.set_style('white')
plt.ylabel('y')
plt.xlabel('x')
plt.axvline(true_mean[0], ls=':')
# plt.axvline(true_mean[0]+1, ls=':')
plt.axhline(true_mean[1], ls=':')
# plt.axhline(0.86+true_mean[1], ls=':')
# plt.colorbar()
# plt.plot(x,x)
# print "data:", data
# plt.contourf(x,y,rv.pdf(grid),25,cmap=cmap)
if True:
# if False:
    plt.scatter(hc_cho_data[0,:],hc_cho_data[1,:], 
                alpha=0.01,
                c='k',
                s=9,
               edgecolor='')
plt.xlim(*pltlims[0])
plt.ylim(*pltlims[1])

In [ ]:
plt.hexbin(hc_cho_data[0,:],hc_cho_data[1,:])
plt.xlim(*pltlims[0])
plt.ylim(*pltlims[1])
plt.colorbar()